#### Auto-fill Questionnaire using Chain of Thought or Few-Shot Examples

This notebook showcases the application of few-shot examples in autofilling questionnaires. It utilizes a json file (`risk_questionnaire.json`) to
provide the LLM with example responses for some use-cases.

By leveraging these few-shot examples, we can enable seamless completion of lengthy questionnaires, minimizing manual effort and improving overall efficiency.


In [1]:
from risk_atlas_nexus.blocks.inference import (
    RITSInferenceEngine,
    WMLInferenceEngine,
    OllamaInferenceEngine,
    VLLMInferenceEngine,
)
from risk_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    RITSInferenceEngineParams,
    WMLInferenceEngineParams,
    OllamaInferenceEngineParams,
    VLLMInferenceEngineParams,
)

from risk_atlas_nexus.data import load_resource
from risk_atlas_nexus.library import RiskAtlasNexus

/Users/dhaval/Projects/Usage-Governance/risk-atlas-nexus/src/risk_atlas_nexus/toolkit/job_utils.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


##### Risk Atlas Nexus uses Large Language Models (LLMs) to infer risks dimensions. Therefore requires access to LLMs to inference or call the model.

**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/risk-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide before going ahead.

_Note:_ RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.


In [2]:
inference_engine = OllamaInferenceEngine(
    model_name_or_path="granite3.2:8b",
    credentials=InferenceEngineCredentials(api_url="OLLAMA_API_URL"),
    parameters=OllamaInferenceEngineParams(
        num_predict=1000, temperature=0, repeat_penalty=1, num_ctx=8192
    ),
)

# inference_engine = WMLInferenceEngine(
#     model_name_or_path="ibm/granite-20b-code-instruct",
#     credentials={
#         "api_key": "WML_API_KEY",
#         "api_url": "WML_API_URL",
#         "project_id": "WML_PROJECT_ID",
#     },
#     parameters=WMLInferenceEngineParams(
#         max_new_tokens=1000, decoding_method="greedy", repetition_penalty=1
#     ),
# )

# inference_engine = VLLMInferenceEngine(
#     model_name_or_path="ibm-granite/granite-3.1-8b-instruct",
#     credentials=InferenceEngineCredentials(
#         api_url="VLLM_API_URL", api_key="VLLM_API_KEY"
#     ),
#     parameters=VLLMInferenceEngineParams(max_tokens=1000, temperature=0.7),
# )

# inference_engine = RITSInferenceEngine(
#     model_name_or_path="ibm-granite/granite-3.1-8b-instruct",
#     credentials={
#         "api_key": "RITS_API_KEY",
#         "api_url": "RITS_API_URL",
#     },
#     parameters=RITSInferenceEngineParams(max_tokens=1000, temperature=0.7),
# )

[2025-05-26 18:30:21:729] - INFO - RiskAtlasNexus - OLLAMA inference engine will execute requests on the server at http://localhost:11434.
[2025-05-26 18:30:21:803] - INFO - RiskAtlasNexus - Created OLLAMA inference engine.


##### Create an instance of RiskAtlasNexus

_Note: (Optional)_ You can specify your own directory in `RiskAtlasNexus(base_dir=<PATH>)` to utilize custom AI ontologies. If left blank, the system will use the provided AI ontologies.


In [3]:
risk_atlas_nexus = RiskAtlasNexus()

[2025-05-26 18:30:21:880] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None


#### Defining Examples for Auto-Assist Functionality

This cell showcases the template used in `risk_atlas_nexus/data/templates/risk_questionnaire.json` to provide questions and their examples for auto-assist functionality.

**Customization:**

To adapt this auto-assist functionality to custom questionnaires, users need to provide their own set of questions, example intents, and corresponding answers in a json file (e.g., `risk_questionnaire.json`). This will enable the LLM to learn from these few-shot examples and generate responses for unseen queries.

**Template Structure: Zero Shot**

Each question is accompanied by corresponding examples provided as an empty list.

```shell
  [
      {
          "question": "In which environment is the system used?",
          "cot_examples": []
      }
      ...
  ]
```

**Template Structure: Few Shot**

Each question is associated with a list of examples, each containing intent, answer, and optional explanation.

```shell
  [
      {
          "question": "In which environment is the system used?",
          "cot_examples": [
            {
              "intent": "Find patterns in healthcare insurance claims",
              "answer": "Insurance Claims Processing or Risk Management or Data Analytics",
              "explanation": "The system might be used by an insurance company's claims processing department to analyze and identify patterns in healthcare insurance claims."
            },
            {
                "intent": "optimize supply chain management in Investment banks",
                "answer": "Treasury Departments or Asset Management Divisions or Private Banking Units",
                "explanation": null
            },
            ...
          ]
      }
      ...
  ]
```

In this notebook, we're using a simplified template to cover 7 questions
from the Airo questionnaire:

1. AI Domain
2. System environment
3. Utilized techniques
4. Intended User
5. Intended Purpose
6. System Application
7. AI Subject


#### Load Risk Questionnaire

**Note:** Please load your custom questionnaire below according to the specified format to utilize custom questionnaire instead.


In [7]:
risk_questionnaire = load_resource("risk_questionnaire.json")

risk_questionnaire[0]

{'question': 'What domain does your use request fall under? Customer service/support, Technical, Information retrieval, Strategy, Code/software engineering, Communications, IT/business automation, Writing assistant, Financial, Talent and Organization including HR, Product, Marketing, Cybersecurity, Healthcare, User Research, Sales, Risk and Compliance, Design, Other',
 'cot_examples': [{'intent': 'Optimize supply chain management in Investment banks',
   'answer': 'Strategy',
   'explanation': 'Since the task is involved in improving the processes to ensure better performance. It is not finance since the task is on supply chain optimization and not on financial aspects even though the application domain is banks.'},
  {'intent': 'Ability to create dialog flows and integrations from natural language instructions.',
   'answer': 'Customer service/support',
   'explanation': 'Since the task relates to human conversations or generating human converstations or support.'},
  {'intent': 'Chec

There are two ways to use the inference engine to get the LLM outputs. `generate_zero_shot_risk_questionnaire_output` which gives the zero-shot output for the question and `generate_few_shot_risk_questionnaire_output` which gives the output using few-shot examples defined above.


#### Auto Assist Questionnaire - Zero Shot


In [5]:
usecase = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."

results = risk_atlas_nexus.generate_zero_shot_risk_questionnaire_output(
    usecase, risk_questionnaire, inference_engine
)

# Display Results
for index, (question_data, result) in enumerate(
    zip(risk_questionnaire, results), start=1
):
    print(
        f"\n{index}: "
        + question_data["question"]
        + "\nA: "
        + result.prediction["answer"]
    )

Inferring with OLLAMA: 100%|██████████| 7/7 [00:24<00:00,  3.46s/it]


1: What domain does your use request fall under? Customer service/support, Technical, Information retrieval, Strategy, Code/software engineering, Communications, IT/business automation, Writing assistant, Financial, Talent and Organization including HR, Product, Marketing, Cybersecurity, Healthcare, User Research, Sales, Risk and Compliance, Design, Other
A: Customer service/support

2: In which environment is the system used?
A: The system is used in a digital environment, specifically designed for online interactions between customers and support agents.

3: What techniques are utilised in the system? Multi-modal: {Document Question/Answering, Image and text to text, Video and text to text, visual question answering}, Natural language processing: {feature extraction, fill mask, question answering, sentence similarity, summarization, table question answering, text classification, text generation, token classification, translation, zero shot classification}, computer vision: {image cl

#### Auto Assist Questionnaire - Few Shot


In [8]:
usecase = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."

results = risk_atlas_nexus.generate_few_shot_risk_questionnaire_output(
    usecase,
    risk_questionnaire,
    inference_engine,
)

# Display Results
for index, (question_data, result) in enumerate(
    zip(risk_questionnaire, results), start=1
):
    print(
        f"\n{index}: "
        + question_data["question"]
        + "\nA: "
        + result.prediction["answer"]
    )

Inferring with OLLAMA: 100%|██████████| 7/7 [00:36<00:00,  5.23s/it]


1: What domain does your use request fall under? Customer service/support, Technical, Information retrieval, Strategy, Code/software engineering, Communications, IT/business automation, Writing assistant, Financial, Talent and Organization including HR, Product, Marketing, Cybersecurity, Healthcare, User Research, Sales, Risk and Compliance, Design, Other
A: Customer service/support

2: In which environment is the system used?
A: Customer Service or Claims Support Departments

3: What techniques are utilised in the system? Multi-modal: {Document Question/Answering, Image and text to text, Video and text to text, visual question answering}, Natural language processing: {feature extraction, fill mask, question answering, sentence similarity, summarization, table question answering, text classification, text generation, token classification, translation, zero shot classification}, computer vision: {image classification, image segmentation, text to image, object detection}, audio:{audio c